In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from ipywidgets import interact

%matplotlib inline

In [2]:
def fix_dataframe(df):
    params = []
    vals = []
    for key, val in df.items():
        if key == "Unnamed: 0":
            continue
        params.append(key)
        vals.append(val)

    dicos = []
    for stuff in zip(*vals):
        others = []
        dico = {}
        for param, val in zip(params, stuff):
            if not param.startswith("eps"):
                dico[param] = val
            else:
                others.append((param, val))

        if not isinstance(dico["embedding_type"], str):
            dico["method"] = dico["binary"].upper()
        else:
            dico["method"] = dico["embedding_type"]
        for param, val in others:
            sup, eps = param.split("_")[1:3]
            eps = float(eps)
            dicos.append({"eps": eps, "AUC": val, "supervised": sup == "s", **dico})

    return pd.DataFrame(dicos)

# csv_file = "csv/results_8_04_2020_readable.csv"
csv_file = "../../results/csv/fgsm_20_04_2020.csv"
df = pd.read_csv(csv_file)
df = fix_dataframe(df)
df = df.loc[df.architecture != "svhn_lenet_bandw2"]

In [3]:
import os


def filter_column(df, column, value):
    return df.loc[np.logical_or(df[column].isnull(), df[column] == value)]


@interact
def plot_results(# architecture=df.architecture.unique(),
                 supervised=[True, False],
                 mlp=[False, True],
                 # sigmoidize=[False, True],
                 # epochs=[50, 200, 250, 300]
                 ):
    subdf = df.copy()
    subdf["mlp"] = subdf["architecture"].apply(lambda arch: "mlp" in arch or "simple_fcn" in arch)
    #for param, val in zip(["supervised", "mlp", # "sigmoidize"
    #                      ],
    #                      [supervised, mlp,
    #                       # sigmoidize
    #                      ]):
    #    subdf = filter_column(subdf, param, val)
    #hue_order = ["PersistentDiagram", "RawGraph", "LID", "MAHALANOBIS"]
    #sns.catplot(data=subdf, x="eps", y="AUC", hue="method", kind="point", col="architecture", col_wrap=2,
    #            ci=None, estimator=max, hue_order=hue_order)
    #plt.tight_layout()
    #fig_file = "%s_fig_sup=%s_mlp=%s.png" % (os.path.basename(csv_file),
    #                                         supervised, mlp)
    #for param, val in zip(["supervised", "mlp"],
    #                      [supervised, mlp]):
    for param, val in zip(["supervised", "mlp", # "sigmoidize"
                          ],
                          [supervised, mlp,
                           # sigmoidize
                          ]):
        subdf = filter_column(subdf, param, val)
    hue_order = ["PersistentDiagram", "RawGraph", "LID", "MAHALANOBIS"]
    sns.catplot(data=subdf, x="eps", y="AUC", hue="method", kind="point", col="architecture", col_wrap=2,
                ci=None, estimator=max, hue_order=hue_order)
    plt.tight_layout()
    fig_file = "%s_fig_sup=%s_mlp=%s.png" % (os.path.basename(csv_file),
                                             supervised, mlp)
    plt.savefig(fig_file, dpi=200, bbox_inches="tight");
    print("Saved %s" % fig_file)

interactive(children=(Dropdown(description='supervised', options=(True, False), value=True), Dropdown(descript…

In [4]:
df.sample(200)

,eps,AUC,supervised,id,embedding_type,architecture,binary,epochs,sigmoidize,aucs_l2_norm,time,method
112,0.40,0.982864,False,15,NaN,fashion_mnist_lenet,lid,200,False,None,57.717314,LID
100,0.40,0.999408,False,1,RawGraph,simple_fcn_mnist,ocsvm_detector,50,True,None,3596.670124,RawGraph
194,0.10,0.826864,False,24,NaN,cifar_lenet,mahalanobis,300,False,None,6584.419592,MAHALANOBIS
52,0.40,0.996816,False,2,PersistentDiagram,mnist_lenet,ocsvm_detector,50,False,None,960.954407,PersistentDiagram
178,0.40,0.999696,False,9,RawGraph,svhn_lenet,ocsvm_detector,250,False,None,7612.683631,RawGraph
...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.01,0.569376,True,18,NaN,cifar_lenet,lid,300,False,None,55.992808,LID
158,0.10,0.841504,False,23,NaN,svhn_lenet,mahalanobis,250,False,None,7430.626311,MAHALANOBIS
131,0.40,0.999504,True,31,PersistentDiagram,fashion_mnist_lenet,ocsvm_detector,200,True,None,2416.926566,PersistentDiagram
71,0.40,1.000000,True,28,RawGraph,mnist_lenet,ocsvm_detector,50,True,None,3762.087287,RawGraph


In [99]:
import glob
import os

dfs = []
for csv_file in glob.glob("/home/elvis/Téléchargements/principal_*.csv"):
    df = pd.read_csv(csv_file)
    method = os.path.basename(csv_file).split("_")[2]
    if method == "lid":
        method = "LID"
    elif method == "mahalanobis":
        method = "Mahalanobis"
    elif method == "rawgraph":
        method = "*Raw activation graph"
    elif method == "prstdgm":
        method = "*Persistence diagrams"
    else:
        method = method.upper()
    df["method"] = method
    dfs.append(df)
df = pd.concat(dfs)
df.method.unique()

array(['LID', '*Persistence diagrams', '*Raw activation graph',
       'Mahalanobis'], dtype=object)

In [100]:
dico = df.to_dict("list")

items = []
for bunch in zip(*dico.values()):
    tmp = dict((key, val) for key, val in zip(dico.keys(), bunch))
    tmp.pop("unsupervised_predictions")
    tmp.pop("supervised_predictions")
    if not isinstance(tmp["supervised_metrics"], str):
        continue
    sup = eval(tmp.pop("supervised_metrics"))
    unsup = eval(tmp.pop("unsupervised_metrics"))
    all_eps = sup.keys()
    for eps in all_eps:
        for bootstrap in ["lower", "middle", "upper"]:
            if bootstrap == "middle":
                metric = "value"
            else:
                metric = "%s_bound" % bootstrap
            items.append({**tmp, "eps": eps,
                          "bootstrap": bootstrap,
                          "sup_auc": sup[eps]["auc"][metric],
                          "unsup_auc": unsup[eps]["auc"][metric]})
df_clean = pd.DataFrame(items)

In [106]:
@interact
def plot_results(metric=["unsup_auc", "sup_auc"],
                 attack_type=["FGSM", "BIM", "DeepFool", "CW"]
                 ):
    sns.catplot(data=df_clean
                .loc[df_clean.attack_type == attack_type],
                col="architecture", col_wrap=3,
                hue_order=["*Persistence diagrams", "*Raw activation graphs",
                           "LID", "Mahalanobis"],
                x="eps", y=metric, hue="method", kind="point");
    plt.tight_layout()
    fig_file = "pngs/fig_%s_%s.png" % (metric, attack_type)
    plt.savefig(fig_file, dpi=200, bbox_inches="tight");
    print("Saved %s" % fig_file)

interactive(children=(Dropdown(description='metric', options=('unsup_auc', 'sup_auc'), value='unsup_auc'), Dro…

In [77]:
df_clean.architecture.unique()

array(['simple_fcn_mnist', 'mnist_lenet', 'fashion_mnist_lenet',
       'svhn_lenet', 'cifar_lenet'], dtype=object)

In [45]:
df.name.unique()

array(['LID', nan, 'Graph', 'Mahalanobis'], dtype=object)

In [94]:
df.sample(10)

,run_id,dataset,perc_of_nn,attack_type,architecture,epochs,batch_size,unsupervised_metrics,name,supervised_metrics,...,time,supervised_predictions,aucs_l2_norm,method,thresholds,ERROR,l2_diff,linf_diff,sigmoidize,gaussian_accuracy
8,8,CIFAR10,NaN,DeepFool,cifar_lenet,300,NaN,{'1.0': {'auc': {'upper_bound': 0.564827395698...,Graph,{'1.0': {'auc': {'upper_bound': 0.672717317769...,...,657.379884,"{'1.0': [[-0.02117083848684365, 0.024601508707...","{'0.28550365260508515': [0.6272105835198685, 0...",prstdgm,0:0_2:0_4:0.3_5:0.3_6:0.3,NaN,"{'1.0': [0.31262852391714996, 0.35042128273350...","{'1.0': [0.0332093909838197, 0.039258790698610...",NaN,NaN
8,8,CIFAR10,NaN,FGSM,cifar_lenet,300,NaN,{'0.01': {'auc': {'upper_bound': 0.56548766357...,Graph,{'0.01': {'auc': {'upper_bound': 0.60920440102...,...,1281.857495,"{'0.01': [[-0.00018520003901922877, 9.22903310...",None,prstdgm,0:0_2:0_4:0.3_5:0.3_6:0.3,NaN,"{'0.01': [0.5537147279962844, 0.55263007518592...","{'0.01': [0.010000000000000009, 0.010000000000...",NaN,NaN
7,7,SVHN,NaN,CW,svhn_lenet,300,NaN,{'1.0': {'auc': {'upper_bound': 0.566042769230...,Mahalanobis,{'1.0': {'auc': {'upper_bound': 0.540757164426...,...,2948.740618,"{'1.0': [[-0.9782109897216651, -0.968491976011...","{'0.21823997476373788': [0.6237948135843484, 0...",mahalanobis,NaN,NaN,NaN,NaN,NaN,0.800
0,0,MNIST,NaN,DeepFool,simple_fcn_mnist,50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,rawgraph,0:0.01_1:0_2:0,Traceback\ \(most\ recent\ call\ last\)\:\\n\ ...,NaN,NaN,True,NaN
4,4,FashionMNIST,NaN,DeepFool,fashion_mnist_lenet,100,NaN,{'1.0': {'auc': {'upper_bound': 0.760267963853...,Graph,{'1.0': {'auc': {'upper_bound': 0.948195075346...,...,2097.371930,"{'1.0': [[1.0138212491739296e-10, 4.5948467253...","{'0.6586436764661381': [0.6881182857142859, 0....",rawgraph,0:0.05_2:0.05_4:0.05_5:0.05,NaN,"{'1.0': [1.2158619283405243, 3.090813588583960...","{'1.0': [0.17884821719342558, 0.55090290457919...",True,NaN
7,7,SVHN,0.1,CW,svhn_lenet,300,300.0,{'1.0': {'auc': {'upper_bound': 0.663557086840...,LID,{'1.0': {'auc': {'upper_bound': 0.800918891676...,...,45.748274,"{'1.0': [[-0.24948494947801836, 0.919773441612...","{'0.21823997476373788': [0.6992237792791915, 0...",lid,NaN,NaN,NaN,NaN,NaN,NaN
8,8,CIFAR10,0.3,DeepFool,cifar_lenet,300,100.0,{'1.0': {'auc': {'upper_bound': 0.629563078162...,LID,{'1.0': {'auc': {'upper_bound': 0.613304749955...,...,34.802932,"{'1.0': [[0.045285616496865466, 0.004889631610...","{'0.28550365260508515': [0.6578339704995889, 0...",lid,NaN,NaN,NaN,NaN,NaN,NaN
8,8,CIFAR10,NaN,FGSM,cifar_lenet,300,NaN,{'0.01': {'auc': {'upper_bound': 0.60082741743...,Mahalanobis,{'0.01': {'auc': {'upper_bound': 0.56368130383...,...,4480.266954,"{'0.01': [[0.07436384207144708, 0.161509979142...",None,mahalanobis,NaN,NaN,NaN,NaN,NaN,0.568
8,8,CIFAR10,NaN,FGSM,cifar_lenet,300,NaN,{'0.01': {'auc': {'upper_bound': 0.53058779147...,Graph,{'0.01': {'auc': {'upper_bound': 0.53689293685...,...,5081.554988,"{'0.01': [[3.266342751828688e-11, 1.1544987188...",None,rawgraph,0:0_2:0_4:0.3_5:0.3_6:0.3,NaN,"{'0.01': [0.5537147279962844, 0.55263007518592...","{'0.01': [0.010000000000000009, 0.010000000000...",True,NaN
3,3,MNIST,NaN,BIM,mnist_lenet,50,NaN,{'0.01': {'auc': {'upper_bound': 0.98762087587...,Graph,"{'0.01': {'auc': {'upper_bound': 1.0, 'value':...",...,2327.050338,"{'0.01': [[1.0000001293085883, 1.0000001304289...",None,rawgraph,0:0.025_2:0.025_4:0.025_5:0.025,NaN,"{'0.01': [0.1911207097620263, 0.20252780597241...","{'0.01': [0.010000000000000009, 0.010000000000...",True,NaN
